# Input MQTT

Reads from MQTT topics in batches or streams

In [ ]:
import os
#os.environ['create_image']='True'

In [ ]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-39
    RUN pip install ipython nbformat paho-mqtt flask
    ADD input-mqtt.ipynb /
    ENTRYPOINT ["ipython","/input-mqtt.ipynb"]
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t claimed-input-mqtt .
    !docker tag claimed-input-mqtt romeokienzler/claimed-input-mqtt
    !docker push romeokienzler/claimed-input-mqtt
else:
    #!pip install paho-mqtt flask
    None

In [ ]:
import logging
import os
import re
import sys
import random
from paho.mqtt import client as mqtt_client
import threading
from flask import Flask
from flask import abort
import queue

In [ ]:
# mqtt port
port = int(os.environ.get('port',1883))

# mqtt server
broker = os.environ.get('broker')

# mqtt topic
topic = os.environ.get('topic')

# mqtt user name
username = os.environ.get('username', '')

# mqtt password
password = os.environ.get('password', '')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

port = int(port)

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)
    # Set Connecting Client ID
    client = mqtt_client.Client(f'python-mqtt-{random.randint(0, 1000)}')
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

In [ ]:
client = connect_mqtt()
q = queue.Queue()

In [ ]:
def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        print("Message arrived:")
        print(msg.payload.decode())
        q.put(msg.payload.decode())
        print("Message put to queue.")
    client.subscribe(topic)
    client.on_message = on_message

subscribe(client)

x = threading.Thread(target=client.loop_forever)
x.start()

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    try:
        print("Request received, polling queue...")
        message = q.get(block=False)
        print("Returning message:")
        print(message)
        return message
    except queue.Empty:
        print('empty queue!')
        abort(404)


app.run(host='0.0.0.0', port=8080)